# BWT(T) Algorithm to retrieve alignments
* Week 3, Fall 2024 Bioinformatics

### Burrows-Wheeler Transform
* Reversible permutation used originally in compression
* Database sequence T = acaacg$

In [1]:
import numpy as np

In [2]:
# 1. 입력 문자열 준비

In [5]:
sequence = input("sequence:")

sequence: acaacg


In [6]:
sequence += "$"

In [7]:
sequence_list = list(sequence)

In [8]:
sequence_list

['a', 'c', 'a', 'a', 'c', 'g', '$']

# 2. 회전 행렬 생성
문자열의 모든 순환(rotation) 버전을 만드는 과정입니다. 순환이란, 문자열의 각 문자가 하나씩 왼쪽으로 밀리고, 맨 앞에 있던 문자가 맨 뒤로 가는 과정을 반복하는 것을 의미합니다.

### 순환(rotation)의 과정

1. 원본 문자열: 
**ATGC$**  (아무 변화 없음, 첫 번째 버전)

2. 한 칸 왼쪽으로 밀기:
맨 앞의 A를 맨 뒤로 보냅니다:
**TGC$A** (두 번째 버전)

3. 다시 한 칸 왼쪽으로 밀기:
T를 맨 뒤로 보냅니다:
**GC$AT** (세 번째 버전)

4. 또 한 칸 왼쪽으로 밀기:
G를 맨 뒤로 보냅니다:
**C$ATG** (네 번째 버전)


5. 마지막으로 한 칸 왼쪽으로 밀기:
C를 맨 뒤로 보냅니다:
**$ATGC** (다섯 번째 버전)

In [9]:
bwm = [] 

for i in range(len(sequence_list)):
    bwm.append(sequence_list.copy())
    first_seq = sequence_list.pop(0)  # 첫 번째 'a'를 제거
    sequence_list.append(first_seq) #지운 'a'를 맨 뒤로 보냅니다

In [10]:
bwm 

[['a', 'c', 'a', 'a', 'c', 'g', '$'],
 ['c', 'a', 'a', 'c', 'g', '$', 'a'],
 ['a', 'a', 'c', 'g', '$', 'a', 'c'],
 ['a', 'c', 'g', '$', 'a', 'c', 'a'],
 ['c', 'g', '$', 'a', 'c', 'a', 'a'],
 ['g', '$', 'a', 'c', 'a', 'a', 'c'],
 ['$', 'a', 'c', 'a', 'a', 'c', 'g']]

In [11]:
bwm = np.array(bwm)

In [12]:
bwm

array([['a', 'c', 'a', 'a', 'c', 'g', '$'],
       ['c', 'a', 'a', 'c', 'g', '$', 'a'],
       ['a', 'a', 'c', 'g', '$', 'a', 'c'],
       ['a', 'c', 'g', '$', 'a', 'c', 'a'],
       ['c', 'g', '$', 'a', 'c', 'a', 'a'],
       ['g', '$', 'a', 'c', 'a', 'a', 'c'],
       ['$', 'a', 'c', 'a', 'a', 'c', 'g']], dtype='<U1')

## 3. 회전 행렬 정렬
생성된 모든 회전을 사전순으로 정렬

In [15]:
bwm[:, 0] 

array(['a', 'c', 'a', 'a', 'c', 'g', '$'], dtype='<U1')

In [18]:
sorted_bwm = bwm[np.lexsort([bwm[:, i] for i in range((bwm.shape[1]-1), -1, -1)])]

In [19]:
sorted_bwm

array([['$', 'a', 'c', 'a', 'a', 'c', 'g'],
       ['a', 'a', 'c', 'g', '$', 'a', 'c'],
       ['a', 'c', 'a', 'a', 'c', 'g', '$'],
       ['a', 'c', 'g', '$', 'a', 'c', 'a'],
       ['c', 'a', 'a', 'c', 'g', '$', 'a'],
       ['c', 'g', '$', 'a', 'c', 'a', 'a'],
       ['g', '$', 'a', 'c', 'a', 'a', 'c']], dtype='<U1')

In [20]:
bwt_first = sorted_bwm[:, 0]
bwt_last = sorted_bwm[:, -1]

In [21]:
bwt_first

array(['$', 'a', 'a', 'a', 'c', 'c', 'g'], dtype='<U1')

In [22]:
bwt_last

array(['g', 'c', '$', 'a', 'a', 'a', 'c'], dtype='<U1')

In [103]:
def burrows_wheeler_transform(sequence):
    sequence += "$"
    sequence_list = list(sequence)
    
    bwm = [] 
    
    for i in range(len(sequence_list)):
        bwm.append(sequence_list.copy())
        first_seq = sequence_list.pop(0)  # 첫 번째 'a'를 제거
        sequence_list.append(first_seq) #지운 'a'를 맨 뒤로 보냅니다
    
    bwm = np.array(bwm)
    
    sorted_bwm = bwm[bwm[:, 0].argsort()]

    bwt_first = sorted_bwm[:, 0]
    bwt_last = sorted_bwm[:, -1]

    return bwt_first, bwt_last

# LF mapping 하기

In [24]:
char_first_loc = {}
for index, char in enumerate(bwt_first):
    if char not in char_first_loc.keys():
        char_first_loc[char] = []
    char_first_loc[char].append(index)    

In [26]:
char_last_loc = {}
for index, char in enumerate(bwt_last):
    if char not in char_last_loc.keys():
        char_last_loc[char] = []
    char_last_loc[char].append(index)        

In [25]:
char_first_loc

{'$': [0], 'a': [1, 2, 3], 'c': [4, 5], 'g': [6]}

In [27]:
char_last_loc

{'g': [0], 'c': [1, 6], '$': [2], 'a': [3, 4, 5]}

In [28]:
t = np.array([])

In [29]:
t

array([], dtype=float64)

In [30]:
t = np.append(t, bwt_last[0])
t

array(['g'], dtype='<U32')

In [83]:
bwt_last[0]

'g'

In [84]:
bwt_last[char_first_loc[bwt_last[0]]]

array(['c'], dtype='<U1')

In [55]:
second_seq_index = char_first_loc[bwt_last[0]]

In [56]:
second_seq_index = int(second_seq_index[0])

In [67]:
second_seq_index

6

In [68]:
bwt_last[second_seq_index]

'c'

In [71]:
char_last_loc[bwt_last[second_seq_index]].index(second_seq_index)

1

In [74]:
char_first_loc[bwt_last[second_seq_index]]

[4, 5]

In [77]:
third_seq_index = char_first_loc[bwt_last[second_seq_index]][char_last_loc[bwt_last[second_seq_index]].index(second_seq_index)]

In [76]:
third_seq_index

5

In [78]:
bwt_last[third_seq_index]

'a'

In [79]:
char_last_loc[bwt_last[third_seq_index]].index(third_seq_index)

2

In [80]:
char_first_loc[bwt_last[third_seq_index]]

[1, 2, 3]

In [81]:
fourth_seq_index = char_first_loc[bwt_last[third_seq_index]][char_last_loc[bwt_last[third_seq_index]].index(third_seq_index)]

In [82]:
fourth_seq_index

3

In [102]:
original_sequence = np.array([])
first_seq_index = 0
seq = bwt_last[first_seq_index] #가장 처음으로 입력되는, 즉 뒤에서 첫번째는 bwt_last의 첫번째 시퀀스
original_sequence = np.append(seq, original_sequence)
seq_last_index = char_last_loc[bwt_last[first_seq_index]].index(first_seq_index)
next_seq_index = char_first_loc[seq][seq_last_index]

print(next_seq_index)

seq = bwt_last[next_seq_index] 
original_sequence = np.append(seq, original_sequence)
seq_last_index = char_last_loc[bwt_last[next_seq_index]].index(next_seq_index)
next_seq_index = char_first_loc[seq][seq_last_index]

print(next_seq_index)
print(original_sequence)

6
5
['c' 'g']


In [104]:
def lf_mapping(bwt_first, bwt_last):
    char_first_loc = {}
    for index, char in enumerate(bwt_first):
        if char not in char_first_loc.keys():
            char_first_loc[char] = []
        char_first_loc[char].append(index) 
        
    char_last_loc = {}
    for index, char in enumerate(bwt_last):
        if char not in char_last_loc.keys():
            char_last_loc[char] = []
        char_last_loc[char].append(index)    
        
    original_sequence = np.array([])
    next_seq_index = 0
    
    for i in range(len(bwt_last)):
        seq = bwt_last[next_seq_index] #가장 처음으로 입력되는, 즉 뒤에서 첫번째는 bwt_last의 첫번째 시퀀스
        original_sequence = np.append(seq, original_sequence)
        seq_last_index = char_last_loc[bwt_last[next_seq_index]].index(next_seq_index)
        next_seq_index = char_first_loc[seq][seq_last_index]
    return(original_sequence)

print(lf_mapping(bwt_first, bwt_last))

['$' 'a' 'c' 'a' 'a' 'c' 'g']


In [105]:
#retrieve_alignment

In [106]:
query = ['a','a','c']

In [147]:
finding = np.array([])

In [114]:
first = query[-1]

In [116]:
first_pool = char_first_loc[query[-1]]

In [117]:
first_pool

[4, 5]

In [126]:
finding = np.array([])

for i in range(len(query)):
    first_search = query[-1]
    first_pool = char_first_loc[first_search]
    
    for next_seq_index in first_pool:
        seq = bwt_first[next_seq_index]
        if seq == query[len(query)-1]:
            finding = np.append(seq, finding)
            break
        else:
            pass
            
    seq_last_index = char_last_loc[bwt_last[next_seq_index]].index(next_seq_index)
    next_seq_index = char_first_loc[seq][seq_last_index]
    print
print(next_seq_index)
    

5


In [127]:
query[-1]

'c'

In [128]:
char_first_loc[query[-1]]

[4, 5]

In [138]:
idx = char_first_loc[query[-1]][0]

In [148]:
finding = np.append(bwt_first[idx],finding)

In [149]:
left = bwt_last[idx]
left

'a'

In [150]:
seq_last_index = char_last_loc[left].index(idx)
next_seq_index = char_first_loc[left][seq_last_index]

In [151]:
bwt_first[next_seq_index]

'a'

In [156]:
seq_last_index

1

In [152]:
if bwt_last[next_seq_index] == query[-2]:
    finding = np.append(bwt_first[next_seq_index],finding)

In [153]:
finding

array(['a', 'c'], dtype='<U32')

In [155]:
bwt_last[next_seq_index] == 

'$'